# Single Modality Analysis of Simulated Dataset 1 (Spatial Patterns of Varying Complexity)

## Loading packages

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import scanpy as sc
from sklearn.metrics.cluster import adjusted_rand_score
import anndata as ad
import os
os.environ['R_HOME'] = 'E:/R-4.3.1'
os.environ['R_USER'] = 'E:/anaconda/lib/site-packages/rpy2'

import sys
sys.path.append(r'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main')
from Model.utils import mclust_R

## Loading Data

In [ ]:
file_fold_1 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Spatial_Scenario_4/simulation4_RNA'
file_fold_2 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Spatial_Scenario_4/simulation4_Protein'

adata_omics_1 = sc.read_h5ad(file_fold_1 + '.h5ad')
adata_omics_2 = sc.read_h5ad(file_fold_2 + '.h5ad')

sc.tl.pca(adata_omics_1)
sc.tl.pca(adata_omics_2)

batch = 3
adata_analysis_1 = adata_omics_1[adata_omics_1.obs['batch'] == batch]
adata_analysis_2 = adata_omics_2[adata_omics_2.obs['batch'] == batch]

## Single Modality Clustering

In [8]:
mclust_R(adata_analysis_1, used_obsm='X_pca', num_cluster=4)
mclust_R(adata_analysis_2, used_obsm='X_pca', num_cluster=4)

fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%


AnnData object with n_obs × n_vars = 400 × 31
    obs: 'Ground Truth', 'batch', 'clusters_mclust'
    var: 'gene_ids', 'feature_types', 'genome', 'mean', 'std'
    uns: 'pca'
    obsm: 'spatial', 'X_pca'
    varm: 'PCs'

In [3]:
adata_result_1_1 = adata_analysis_1
adata_result_1_2 = adata_analysis_2

In [6]:
adata_result_2_1 = adata_analysis_1
adata_result_2_2 = adata_analysis_2

In [9]:
adata_result_3_1 = adata_analysis_1
adata_result_3_2 = adata_analysis_2

In [10]:
adata_results_1 = adata_result_1_1.concatenate(adata_result_2_1, adata_result_3_1, batch_key='batch')
adata_results_1.obs.rename(columns={'clusters_mclust': 'Modality1'}, inplace=True)
adata_results_1.obsm['Modality1'] = adata_results_1.obsm.pop('X_pca')

adata_results_2 = adata_result_1_2.concatenate(adata_result_2_2, adata_result_3_2, batch_key='batch')
adata_results_2.obs.rename(columns={'clusters_mclust': 'Modality2'}, inplace=True)
adata_results_2.obsm['Modality2'] = adata_results_2.obsm.pop('X_pca')

## Storing the Results

In [13]:
results = sc.read_h5ad('D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Spatial_Scenario_4.h5ad')
results

AnnData object with n_obs × n_vars = 1200 × 3031
    obs: 'Ground Truth', 'batch', 'SpaGCN', 'SpatialGlue', 'STAGATE', 'MultiMAP', 'MultiVI', 'Modality1', 'Modality2'
    obsm: 'Modality1', 'Modality2', 'MultiMAP', 'MultiVI', 'STAGATE', 'SpatialGlue', 'spatial'

In [12]:
results.obs['Modality1'] = adata_results_1.obs['Modality1'].values
results.obsm['Modality1'] = adata_results_1.obsm['Modality1']
results.obs['Modality2'] = adata_results_2.obs['Modality2'].values
results.obsm['Modality2'] = adata_results_2.obsm['Modality2']
results.write_h5ad('D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Spatial_Scenario_4.h5ad')